In [1]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!pip install bqplot

In [ ]:
import pandas as pd
import numpy as np

symbol = "Security 1"
symbol2 = "Security 2"

In [ ]:
price_data = pd.DataFrame(
    np.cumsum(np.random.randn(150, 2).dot([[0.5, 0.4], [0.4, 1.0]]), axis=0) + 100,
    columns=["Security 1", "Security 2"],
    index=pd.date_range(start="01-01-2007", periods=150),
)

dates_actual = price_data.index.values
prices = price_data[symbol].values

In [ ]:
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector,
    IndexSelector,
    BrushIntervalSelector,
    BrushSelector,
    MultiSelector,
    LassoSelector,
    PanZoom,
    HandDraw,
)
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML

In [ ]:
## First we define a Figure
dt_x_fast = DateScale()
lin_y = LinearScale()

x_ax = Axis(label="Index", scale=dt_x_fast)
x_ay = Axis(label=(symbol + " Price"), scale=lin_y, orientation="vertical")
lc = Lines(
    x=dates_actual, y=prices, scales={"x": dt_x_fast, "y": lin_y}, colors=["orange"]
)
lc_2 = Lines(
    x=dates_actual[50:],
    y=prices[50:] + 2,
    scales={"x": dt_x_fast, "y": lin_y},
    colors=["blue"],
)

In [ ]:
## Next we define the type of selector we would like
intsel_fast = FastIntervalSelector(scale=dt_x_fast, marks=[lc, lc_2])

In [ ]:
## Now, we define a function that will be called when the FastIntervalSelector is interacted with
def fast_interval_change_callback(change):
    db_fast.value = "The selected period is " + str(change.new)

In [ ]:
## Now we connect the selectors to that function
intsel_fast.observe(fast_interval_change_callback, names=["selected"])

In [ ]:
## We use the HTML widget to see the value of what we are selecting and modify it when an interaction is performed
## on the selector
db_fast = HTML()
db_fast.value = "The selected period is " + str(intsel_fast.selected)

fig_fast_intsel = Figure(
    marks=[lc, lc_2],
    axes=[x_ax, x_ay],
    title="Fast Interval Selector Example",
    interaction=intsel_fast,
)  # This is where we assign the interaction to this particular Figure

VBox([db_fast, fig_fast_intsel])

In [ ]:
db_index = HTML(value="[]")

In [ ]:
## Now we try a selector made to select all the y-values associated with a single x-value
index_sel = IndexSelector(scale=dt_x_fast, marks=[lc, lc_2])

In [ ]:
## Now, we define a function that will be called when the selectors are interacted with
def index_change_callback(change):
    db_index.value = "The selected date is " + str(change.new)

In [ ]:
index_sel.observe(index_change_callback, names=["selected"])

In [ ]:
fig_index_sel = Figure(
    marks=[lc, lc_2],
    axes=[x_ax, x_ay],
    title="Index Selector Example",
    interaction=index_sel,
)
VBox([db_index, fig_index_sel])

In [ ]:
from datetime import datetime as py_dtime

dt_x_index = DateScale(min=np.datetime64(py_dtime(2006, 6, 1)))
lin_y2 = LinearScale()

lc2_index = Lines(x=dates_actual, y=prices, scales={"x": dt_x_index, "y": lin_y2})

x_ax1 = Axis(label="Date", scale=dt_x_index)
x_ay2 = Axis(label=(symbol + " Price"), scale=lin_y2, orientation="vertical")

In [ ]:
intsel_date = FastIntervalSelector(scale=dt_x_index, marks=[lc2_index])

In [ ]:
db_date = HTML()
db_date.value = str(intsel_date.selected)

In [ ]:
## Now, we define a function that will be called when the selectors are interacted with - a callback
def date_interval_change_callback(change):
    db_date.value = str(change.new)

In [ ]:
## Notice here that we call the observe on the Mark lc2_index rather than on the selector intsel_date
lc2_index.observe(date_interval_change_callback, names=["selected"])

fig_date_mark = Figure(
    marks=[lc2_index],
    axes=[x_ax1, x_ay2],
    title="Fast Interval Selector Selected Indices Example",
    interaction=intsel_date,
)

VBox([db_date, fig_date_mark])

In [ ]:
date_fmt = "%m-%d-%Y"

sec2_data = price_data[symbol2].values
dates = price_data.index.values

In [ ]:
sc_x = LinearScale()
sc_y = LinearScale()

scatt = Scatter(x=prices, y=sec2_data, scales={"x": sc_x, "y": sc_y})

sc_xax = Axis(label=(symbol), scale=sc_x)
sc_yax = Axis(label=(symbol2), scale=sc_y, orientation="vertical")

In [ ]:
br_sel = BrushSelector(x_scale=sc_x, y_scale=sc_y, marks=[scatt], color="red")

db_scat_brush = HTML(value="[]")

In [ ]:
## call back for the selector
def brush_callback(change):
    db_scat_brush.value = str(br_sel.selected)

In [ ]:
br_sel.observe(brush_callback, names=["brushing"])

In [ ]:
fig_scat_brush = Figure(
    marks=[scatt],
    axes=[sc_xax, sc_yax],
    title="Scatter Chart Brush Selector Example",
    interaction=br_sel,
)

In [ ]:
VBox([db_scat_brush, fig_scat_brush])

In [ ]:
sc_brush_dt_x = DateScale(date_format=date_fmt)
sc_brush_dt_y = LinearScale()

scatt2 = Scatter(
    x=dates_actual, y=sec2_data, scales={"x": sc_brush_dt_x, "y": sc_brush_dt_y}
)

In [ ]:
br_sel_dt = BrushSelector(x_scale=sc_brush_dt_x, y_scale=sc_brush_dt_y, marks=[scatt2])

In [ ]:
db_brush_dt = HTML(value=str(br_sel_dt.selected))

In [ ]:
## call back for the selector
def brush_dt_callback(change):
    db_brush_dt.value = str(br_sel_dt.selected)

In [ ]:
br_sel_dt.observe(brush_dt_callback, names=["brushing"])

In [ ]:
sc_xax = Axis(label=(symbol), scale=sc_brush_dt_x)
sc_yax = Axis(label=(symbol2), scale=sc_brush_dt_y, orientation="vertical")
fig_brush_dt = Figure(
    marks=[scatt2],
    axes=[sc_xax, sc_yax],
    title="Brush Selector with Dates Example",
    interaction=br_sel_dt,
)

In [ ]:
VBox([db_brush_dt, fig_brush_dt])